<a href="https://colab.research.google.com/github/2653319/Sinosteel-Character-Recognition/blob/main/img_capture_%E7%B7%A8%E8%99%9F%E8%BE%A8%E8%AD%98_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import cv2
import numpy as np
from PIL import Image 
%matplotlib inline
#!pwd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_train = pd.read_csv('public_training_data.csv') #訓練CSV
df_train

,filename,label,top right x,top right y,bottom right x,bottom right y,bottom left x,bottom left y,top left x,top left y
0,oR72wdOuzdHVh3sMMmC4gMG2aG5jeub,3LN996012F,706.84430,411.87338,707.51070,455.82960,371.28748,462.13885,370.62106,418.18262
1,7VVWhb6tKde=x=w5x6_p=lfcTlGTBa,2FV271041L,767.03550,426.20898,767.60080,488.00024,428.13678,491.84418,427.57147,430.05290
2,VweTPTTYjERnFWFr3R7=YFKrj9NBT1Lg,2FV413012A,840.83276,495.59230,841.15540,551.61390,421.96686,554.60376,421.64413,498.58215
3,XOIg4ZPERXLzBmsdMd4XNHYH1r=Urbr,3LP563013A,871.62260,522.91460,872.52136,593.15930,352.70407,601.39160,351.80527,531.14690
4,jm85agd8RyzlDBl6jjnpypYIuVw7BTvb,3LR436033F,837.74120,424.97200,838.30910,474.88922,469.82086,480.07870,469.25296,430.16144
...,...,...,...,...,...,...,...,...,...,...
12062,KVbBuo6P=Pv4FsTMGW84LHFgiK2H2=nU,3LZ146042A,879.31635,445.65146,880.98413,500.45438,487.31512,512.43494,485.64730,457.63205
12063,mVE9bWuj9S_vFwQHEIH_e5mMZ4javQF8,2G2204042K,873.36597,478.13287,874.51770,526.46520,477.86438,535.91770,476.71262,487.58540
12064,glBWhEUeVftedEqw=qq=mzBrC=gwMyFs,2FM573063K,852.99220,498.73690,853.77070,557.14670,414.06592,563.00720,413.28740,504.59738
12065,Xv19H4GQWajfdKTsHBcJoz8RaKTVa7BR,1WG43V203,762.74390,447.21410,763.09863,496.17346,475.32275,498.25880,474.96802,449.29944


In [ ]:
df_path = pd.DataFrame('drive/MyDrive/new_public_training_data/' + df_train['label'] + '.jpg') #訓練圖片

In [ ]:
df_path['label'][0]

'drive/MyDrive/new_public_training_data/3LN996012F.jpg'

In [ ]:
# for h in range(len(df_path)):  #縮放全部圖片
#     img = cv2.cvtColor(cv2.imread(df_path['label'][h]), cv2.COLOR_BGR2RGB)
#     image = cv2.resize(img, (256, 256), interpolation=cv2.INTER_AREA)   
#     if(h % 10 == 0):
#         print("h:" + str(h))
#     cv2.imwrite('drive/MyDrive/Resize_public_training_data/' + df_train['label'][h] + '.jpg', image)  #寫入檔案

In [ ]:
df_cut = pd.DataFrame('drive/MyDrive/Resize_public_training_data/' + df_train['label'] + '.jpg')

In [ ]:
img_array = np.zeros((len(df_cut), 256,256), dtype=np.int)

In [ ]:
for c in range(len(df_cut)):
  I = Image.open(df_cut['label'][c])
  L = I.convert('L')
  img = np.array(L)
  img_array[c] = img

In [ ]:
# img_array = np.expand_dims(img_array, axis=-1)
img_array = img_array.reshape(len(img_array), 65536).astype('int')

In [ ]:
len(img_array)

In [ ]:
img_array.shape  #題目

In [ ]:
dict = ['Nashi','A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','0','1','2','3','4','5','6','7','8','9']

In [ ]:
leng = []  #查看最大值用來建立字典
for z in range(len(df_train)):
  leng.append(len(df_train['label'][z]))
z = max(leng)

In [ ]:
dict_str = np.zeros(((len(df_train)), z,37), dtype=np.int)
# dict_str = np.empty((len(df_train), z ),dtype=object)

In [ ]:
for i in range(len(df_train)):   #將答案根據字典轉換
  str1 = list(df_train['label'][i])
  for j in range(len(str1)):
    for k in range(37):
      if(str1[j] == dict[k]):
        dict_str[i][j][k] = 1
        break

In [ ]:
dict_str.shape  #答案

In [ ]:
dict_str_2 = dict_str.reshape(len(dict_str), 444).astype('int')

In [ ]:
# import sys  #顯示完整陣列
# np.set_printoptions(threshold=sys.maxsize)
# # print(img_array[0])

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

In [ ]:
model = Sequential()

In [ ]:
# model.add(Flatten(input_shape = (256,256)))

In [ ]:
model.add(Dense(units=1000, input_dim = 65536 , kernel_initializer = 'normal', activation='relu'))

In [ ]:
model.add(Dropout(0.5))

In [ ]:
model.add(Dense(units = 256 , kernel_initializer = 'normal' , activation = 'relu'))

In [ ]:
model.add(Dropout(0.5))

In [ ]:
model.add(Dense(units=444, kernel_initializer = 'normal', activation='softmax'))

In [ ]:
print(model.summary())

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

In [ ]:
train_history = model.fit(x = img_array ,y = dict_str_2 ,validation_split=0.2,epochs=100, batch_size=120,verbose=2)

In [ ]:
prediction=model.predict(img_array)

In [ ]:
prediction.shape

In [ ]:
prediction = prediction.reshape(len(prediction), 12,37).astype('float32')

In [ ]:
prediction.shape

In [ ]:
prediction[0]

In [ ]:
dict_ToAns = np.empty((len(prediction)) ,dtype=object)  

In [ ]:
for c in range(len(prediction)): #12067
  str3 = ""
  for b in range(len(dict_str[c])):  #12
    for d in range(len(dict)):   #37
      if(prediction[c][b][d]!=0):
        str3 = str3 + dict[d]
  dict_ToAns[c] = str3